In [107]:
#Notebook which provides best troughput achievable for a given M, N and d with existence of each edge being variable
#I'll probably make this into a proper python file soon, but it's more convenient this way for now
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import numpy as np

In [108]:
N = 16
dE = 8

In [109]:
#Specify the demand matrix
# demand = np.zeros((N,N))
# for i in range(N):
#     for j in range(N):
#         if i!=j:
#             demand[i][j] = dE/(N-1)


workdir="/home/studium/Documents/Code/rdcn-throughput/matrices/"
demand= np.loadtxt(workdir+"skew-16-0.9.mat", usecols=range(N))
demand = demand * dE

In [110]:
#Initialize the model
model = gp.Model("throughput")
#TODO: Learn more about tolerances and which to use
# model.Params.OptimalityTol = 1e-9
# model.Params.FeasibilityTol = 1e-3


In [111]:
#Add edge vars
edge_vars = {}
for i in range(N):
    for j in range(N):
        edge_vars[i, j] = model.addVar(vtype= GRB.INTEGER, name=f"edge_{i}_{j}",ub=dE)

In [112]:
#Inc edges constraint
for i in range(N):
    model.addConstr(gp.quicksum(edge_vars[i,j] for j in range(N))<=dE)

In [113]:
#Outg edges constraint
for i in range(N):
    model.addConstr(gp.quicksum(edge_vars[j,i] for j in range(N))<=dE)

In [114]:
#Add flow vars for every node combination
flow_vars = {}
for i in range(N):
    for j in range(N):
        for s in range(N):
            for d in range(N):
                flow_vars[s, d, i, j] = model.addVar(vtype= GRB.CONTINUOUS, name=f"flow_{s}_{d}_{i}_{j}",lb=0)

In [115]:
#Add throughput variable
throughput = model.addVar(vtype=GRB.CONTINUOUS,name='throughput',lb=0,ub=1)

In [116]:
#Source demand constraint
for s in range(N):
    for d in range(N):
            model.addConstr(gp.quicksum(flow_vars[s, d, s, i] for i in range(N) if i!=s )>= demand[s,d]* throughput,name =f"sdconst_{s}_{d}")

In [117]:
#Dest demand constraint
for s in range(N):
    for d in range(N):
            model.addConstr(gp.quicksum(flow_vars[s, d, i, d] for i in range(N) if i!=d )>= demand[s,d]* throughput,name =f"ddconst_{s}_{d}")

In [118]:
#Flow conservation
for s in range(N):
    for d in range(N):
        for j in (j for j in range(N) if j != s and j != d):
            model.addConstr(gp.quicksum(flow_vars[s, d, i, j] for i in range(N) if i!=j )-gp.quicksum(flow_vars[s, d, j, k] for k in range(N) if k!=j)== 0,name =f"fcconst_{s}_{d}_{j}")

In [119]:
#Capacity Constraints
for i in range(N):
    for j in range(N):
        model.addConstr(gp.quicksum(gp.quicksum(flow_vars[s,d,i,j] for d in range(N)) for s in range(N))<= edge_vars[i,j],name =f"cconst_{s}_{d}")

In [120]:
#Set the objective: maximize throughput
model.setObjective(throughput, GRB.MAXIMIZE)

In [ ]:
model.optimize()
print("Throughput for the given topology and the given demand matrix is:", throughput.X)
for v in model.getVars():
        if v.x != 0:
                print(v.varName, "=", v.x)